In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from pylab import rcParams


In [2]:
class Profile():
    def __init__(self, filename):
        self._metadata = {}
        self._projects = {}
        self._votes = {}
        with open(filename, "r", encoding="utf8") as self.file:
            self.__read_lines()

        self.__convert_projects()
        self.__convert_votes()
        
    @property
    def ballots(self):
        return self._ballots
        
    def __convert_projects(self):
        self._projectid_to_index = {}
        tmp = {}
        for i, (proj_id, budget) in enumerate(self._projects.items()):
            self._projectid_to_index[proj_id] = i
            tmp[i] = budget
        self._projects = tmp
        
    def __convert_votes(self):
        self._votes = [np.array([self._projectid_to_index[x]]) if isinstance(x, int) else np.array([self._projectid_to_index[int(y)] for y in x.split(",")]) for x in self._votes.values()]
        self._ballots = np.zeros((self._metadata["num_votes"], self._metadata["num_projects"]))
        for i, vote in enumerate(self._votes):
            self._ballots[i,vote] = 1
       
    def __read_lines(self):
        _sections = {"META":self._metadata, 
                     "PROJECTS":self._projects, 
                     "VOTES":self._votes}
        _slices = {"key":"value",
                   "project_id":"cost",
                   "voter_id":"vote"}
        
        for line in self.file:
            line = line.strip()

            items = line.split(";")
            # find the right index for one of the properties (value, cost, vote)
            try:
                index = items.index(_slices[items[0]])
            except KeyError:
                pass
            else:
                continue 
            
            # switch to a new dict when a new section is found
            try: 
                _current = _sections[line]
            except KeyError:
                pass
            else:
                continue
            
            # read data
            try:
                try:
                    key = int(items[0])
                except ValueError:
                    key = items[0]
                _current[key] = int(items[index])
            except IndexError:
                pass
            except ValueError:
                _current[key] = items[index]

In [3]:
with open("data//sorted_files.pkl", 'rb') as f:
    file_division = pickle.load(f)

In [4]:
approval_profile_files = sorted(file_division['approval'], key=lambda x: x[2], reverse=True)

In [11]:
x = Profile(approval_profile_files[0][0])
x._metadata
# x._projects

{'description': 'District PB in Warsaw, Praga-Południe',
 'country': 'Poland',
 'unit': 'Warszawa',
 'subunit': 'Praga-Południe',
 'instance': 2018,
 'district': 'Praga-Południe',
 'num_projects': 35,
 'num_votes': 8700,
 'budget': '707607,00',
 'vote_type': 'approval',
 'rule': 'greedy',
 'date_begin': '14.06.2017',
 'date_end': '30.06.2017',
 'min_length': 1,
 'max_sum_cost': '707607,00',
 'language': 'polish',
 'edition': 4}

In [10]:
x.ballots

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])